In [1]:
import asyncio
import aiohttp
import json
from pprint import pprint

from attraction_api import fetch_attractions
from hotel_api import HotelRetriever
from flight_api import FlightRetriever
from llm_retrieve import llm_retrieve

In [2]:
itinerary = llm_retrieve()
print("Loaded cities and parameters:")
for city, info in itinerary.items():
    print(f" • {city}: Arrival={info['Arrival Date']}, Departure={info['Departure Date']}, Airport={info['Airport']}")



Loaded cities and parameters:
 • Srinagar: Arrival=2025-07-12, Departure=2025-07-14, Airport=SXR
 • Leh: Arrival=2025-07-14, Departure=2025-07-16, Airport=IXL
 • Shimla: Arrival=2025-07-16, Departure=2025-07-19, Airport=SLV


In [3]:
async def gather_all():
    async with aiohttp.ClientSession() as session:
        attraction_tasks = [fetch_attractions(session, city) for city in itinerary.keys()]
        attraction_results = await asyncio.gather(*attraction_tasks)

    hotel_results = HotelRetriever(itinerary, numadults=2)
    flight_results = FlightRetriever(itinerary)

    return attraction_results, hotel_results, flight_results


In [4]:
# Cell 4: Run & Merge
attractions, hotels, flights = await gather_all()

# Merge results by city
combined = {city: {"city": city, "attractions": [], "hotels": [], "flights": []} for city in itinerary}

# Add attractions
for entry in attractions:
    combined[entry["city"]]["attractions"] = entry["attractions"]

# Add hotels
for entry in hotels:
    city = entry["city"]
    combined[city]["hotels"] = entry["hotels_by"]

# Add flights
for entry in flights:
    from_city = entry["from"]
    to_city = entry["to"]
    date = entry["date"]
    flights_data = entry.get("flights")
    note = entry.get("note")

    # Associate the flight with departure city
    combined[from_city].setdefault("flights", []).append({
        "to": to_city,
        "date": date,
        "flights": flights_data,
        "note": note
    })

# Add user input fields after merging
traveller_count = 4  # set manually or collect from UI
for city, meta in itinerary.items():
    if city in combined:
        combined[city]["arrival_date"] = meta["Arrival Date"]
        combined[city]["departure_date"] = meta["Departure Date"]
        combined[city]["airport"] = meta["Airport"]
        combined[city]["travellers"] = traveller_count
    

[Attraction] 🔍 Fetching location_id for: Srinagar
[Attraction] 🔍 Fetching location_id for: Leh
[Attraction] 🔍 Fetching location_id for: Shimla


[Attraction] ✅ location_id for Srinagar = 297623
[Attraction] 🔍 Fetching attractions for Srinagar


[Attraction] ✅ location_id for Shimla = 304552
[Attraction] 🔍 Fetching attractions for Shimla


[Attraction] ✅ location_id for Leh = 4266551
[Attraction] 🔍 Fetching attractions for Leh


[Attraction] 🌐 Raw results count for Srinagar: 20
[Attraction] 🎯 Returning 10 attractions for Srinagar


[Attraction] 🌐 Raw results count for Shimla: 20
[Attraction] 🎯 Returning 10 attractions for Shimla


[Attraction] 🌐 Raw results count for Leh: 20
[Attraction] 🎯 Returning 10 attractions for Leh


In [5]:
final_list = list(combined.values())

print("\n\u2705 After merging, sample output for first city:")
print(json.dumps(final_list[0], indent=2))


✅ After merging, sample output for first city:
{
  "city": "Srinagar",
  "attractions": [
    {
      "name": "Dal Lake",
      "num_reviews": "4329",
      "rating": "4.3",
      "ranking_position": "6",
      "neighborhood_info": null,
      "description": "",
      "subcategory": [
        {
          "key": "57",
          "name": "Nature & Parks"
        }
      ]
    },
    {
      "name": "Indira Gandhi Tulip Garden",
      "num_reviews": "1146",
      "rating": "4.6",
      "ranking_position": "2",
      "neighborhood_info": null,
      "description": "",
      "subcategory": [
        {
          "key": "57",
          "name": "Nature & Parks"
        }
      ]
    },
    {
      "name": "Sonamarg Hill Station",
      "num_reviews": "1174",
      "rating": "4.4",
      "ranking_position": "4",
      "neighborhood_info": null,
      "description": "",
      "subcategory": [
        {
          "key": "57",
          "name": "Nature & Parks"
        }
      ]
    },
    {
     

In [6]:
import pandas as pd
from IPython.display import display

# Flatten into a DataFrame (one row per city)
df = pd.json_normalize(final_list, sep="_")
display(df)

# Save JSON
out_path = "combined_data_hotel_attraction_flight_Apr_19th.json"
with open(out_path, "w") as f:
    json.dump(final_list, f, indent=2)

print("\u2705 Saved combined results to", out_path)

,city,attractions,flights,arrival_date,departure_date,airport,travellers,hotels_bayesian_review_score,hotels_popularity,hotels_price
0,Srinagar,"[{'name': 'Dal Lake', 'num_reviews': '4329', '...","[{'to': 'Leh', 'date': '2025-07-14', 'flights'...",2025-07-12,2025-07-14,SXR,4,"[{'name': 'John's Homestay', 'price': 55.1}, {...","[{'name': 'Houseboat Zaindari Palace', 'price'...","[{'name': 'LalaApartment', 'price': 13.07}, {'..."
1,Leh,"[{'name': 'Coca-Cola Park', 'num_reviews': '64...","[{'to': 'Shimla', 'date': '2025-07-16', 'fligh...",2025-07-14,2025-07-16,IXL,4,"[{'name': 'Hyatt Place Salt Lake City/Lehi', '...","[{'name': 'Hyatt Place Salt Lake City/Lehi', '...","[{'name': 'Motel 6 Lehi, UT - Thanksgiving Poi..."
2,Shimla,"[{'name': 'Jakhu Hanuman Temple', 'num_reviews...",[],2025-07-16,2025-07-19,SLV,4,"[{'name': 'Wildflower Hall, An Oberoi Resort, ...",[{'name': 'Victory Hotel and Restaurant by GS ...,"[{'name': 'Staynest Mashobra with balcony', 'p..."


✅ Saved combined results to combined_data_hotel_attraction_flight_Apr_19th.json
